In [ ]:
import serial


class Sender:
    TERMINATOR = '\r'.encode('UTF8')

    def __init__(self, device='/dev/ttyACM0', baud=9600, timeout=1):
        self.serial = serial.Serial(device, baud, timeout=timeout)
       
    def receive(self) -> str:
        line = self.serial.read_until(self.TERMINATOR)
        return line.decode('UTF8').strip()

    def send(self, text: str) -> str:
        line = '%s\r\f' % text
        self.serial.write(line.encode('UTF8'))
        # the line should be echoed.
        # If it isn't, something is wrong.
        return self.receive()

    def close(self):
        self.serial.close()

In [ ]:
s=Sender("COM4") #set up Pi Pico

In [ ]:
s.send("1+1") #check serial is working

In [ ]:
s.receive() #should give back 2

In [ ]:
s.send("initialise()") #must be called before moving

In [ ]:
s.send("getcurrentpos()") #current position of the receiver in steps from the limit switch


In [ ]:
#imports for the picoscope

from __future__ import division
from __future__ import absolute_import
from __future__ import print_function
from __future__ import unicode_literals

import time
from time import sleep
from picoscope import ps2000
from picoscope import ps2000a
import matplotlib.pyplot as plt
import numpy as np


print(__doc__)

In [ ]:
#set up picoscope

ps = ps2000.PS2000()

waveform_desired_duration = 50E-6
obs_duration = 3 * waveform_desired_duration #range plotted
sampling_interval = obs_duration / 4096 #sampling interval

(actualSamplingInterval, nSamples, maxSamples) = \
    ps.setSamplingInterval(sampling_interval, obs_duration)

In [ ]:
import scipy.optimize
from scipy.optimize import curve_fit


def sinsignalfunc(t,A,f,psi,x):
    return A*np.sin(2*np.pi*f*t+psi)-x

#fitting function


In [ ]:
#example function to collect data for edge diffraction z'=69cm, z=30cm
#including standard deviation of traces 
#moves 1mm every time for a total of 35cm

#set up dictionary for results

edge_30692={}
edge_30692["position"] = []
edge_30692["channelA"] = []
edge_30692["channelB"] = []  #channel B recevier data in this case

edge_30692["std"] = []
#5cm behind screen is 30000 steps from limit, want to go 40 cm total, 397 steps is 1mm
i=0
while i<350:
   
    edge_30692["position"].append(i*400)
    
    fullarrayA=[]
    fullarrayB=[]
    currentiteration=0
    totaliterations=100    #100 traces
    
    while currentiteration < totaliterations:
        ps.setChannel('A', 'DC', 10.0, 0.0, enabled=True,BWLimited=False)
    #print("Chosen channel range = %d" % channelRange)
    
        ps.setChannel('B', 'DC', 10.0, 0.0, enabled=True,BWLimited=False)
    
        ps.setSimpleTrigger('A', 1.0, 'Falling', timeout_ms=100, enabled=True) #setting a trigger on channel A, square waveform
    
        ps.runBlock()
        ps.waitReady()
        dataA = ps.getDataV('A', nSamples, returnOverflow=False)     #collecting data for both channels 
        dataB = ps.getDataV('B', nSamples, returnOverflow=False)
        fullarrayA.append(list(dataA))
        fullarrayB.append(list(dataB))
        currentiteration+=1
        
    dataTimeAxis = np.arange(nSamples) * actualSamplingInterval
    edge_30692["std"].append(np.std(np.asarray(fullarrayB),axis=0))
    
    edge_30692["channelA"].append(np.mean(np.asarray(fullarrayA),axis=0))
    edge_30692["channelB"].append(np.mean(np.asarray(fullarrayB),axis=0))
    s.send("move(400,0)")
    sleep(1)
    i+=1
#-5cm to 30 cm, move 35cm 400=1mm = 140000 steps 

In [ ]:
#plotting amplitudes

edge_30692amplitudes=[]
edge_30692covariance=[]

for i in range(350):
    popt, pcov= scipy.optimize.curve_fit(sinsignalfunc,np.arange(0,3750),edge_30692["channelB"][i],p0=[1,0.0016,1,0])
    edge_30692amplitudes.append(popt[0])
    edge_30692covariance.append(np.sqrt(pcov[0][0]))
    
plt.plot(np.arange(-5,30,0.1),(np.abs(np.asarray(edge_30692amplitudes))),"r.")